In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd

def finres(npc_input, ins_score_input):
    # Load and prepare tables
    npc_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\npc.csv")
    npc_table = npc_table.fillna("NA")
    ins_score_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\insurance_score.csv")
    ins_score_table = ins_score_table.fillna("NA")
    FINRES_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\FIN RES.csv")
    FINRES_table = FINRES_table.fillna("NA")

    # Find the NPC value
    npc_row = npc_table[npc_table['USAA Group Non-payment Cancellations (NPC)'] == npc_input]
    npc_value = npc_row['Value'].values[0] 
    
    # Find the Insurance Score value
    ins_score_row = ins_score_table[ins_score_table['Insurance Score (ChoicePoint AttractTM One)'] == ins_score_input]
    ins_score_value = ins_score_row['Value'].values[0] 

   
    # Find the FIN RES value in the FINRES_table based on npc_value and ins_score_value
    finres_row = FINRES_table[(FINRES_table['NPC'] == npc_value) & (FINRES_table['Ins Score'] == ins_score_value)]
    finres_value = finres_row['FIN RES'].values[0] 
    
    return finres_value


def relationship(pc_col, enterprise_col, enterprise_ten, business_type_input):
    pc_col_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\P & C Collateral (Non-Auto, Non-Homeowners).csv")
    enterprise_col_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\enterprise_collateral.csv")
    enterprise_ten_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\enterprise tenure.csv")
    enterprise_ten_table= enterprise_ten_table.fillna("None")
    business_type_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\business type.csv")
    relationship_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\RELATIONSHIP.csv")

    # Lookup in pc_col_table
    pc_col_row = pc_col_table[pc_col_table['P & C Collateral (Non-Auto, Non-Homeowners)'] == pc_col]
    pc_col_value = pc_col_row['Value'].values[0]

    # Lookup in enterprise_col_table
    enterprise_col_row = enterprise_col_table[enterprise_col_table['Enterprise Collateral'] == enterprise_col]
    enterprise_col_value = enterprise_col_row['Value'].values[0] 

    # Lookup in enterprise_ten_table
    enterprise_ten_row = enterprise_ten_table[enterprise_ten_table['Enterprise Tenure'] == enterprise_ten]
    enterprise_ten_value = enterprise_ten_row['Value'].values[0]

    # Lookup in business_type_table
    business_type_row = business_type_table[business_type_table['Business Type'] == business_type_input]
    business_type_value = business_type_row['Value'].values[0] 

    
    # Find the RELATIONSHIP value in relationship_table
    relationship_row = relationship_table[
        (relationship_table['P & C Collateral (Non-Auto, Non-Homeowners)'] == pc_col_value) &
        (relationship_table['Enterprise Collateral'] == enterprise_col_value) &
        (relationship_table['Enterprise Tenure'] == enterprise_ten_value) &
        (relationship_table['Business Type'] == business_type_value)
    ]

    # Retrieve the RELATIONSHIP value
    relationship_value = relationship_row['RELATIONSHIP'].values[0] 
    
    return relationship_value


def tier(npc_input, ins_score_input, pc_col, enterprise_col, enterprise_ten, business_type_input):
    # Load the TIER table
    tier_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\tier\\TIER.csv")

    # Get the FINRES value using finres function
    finres_value = finres(npc_input, ins_score_input)

    # Get the RELATIONSHIP value using relationship function
    relationship_value = relationship(pc_col, enterprise_col, enterprise_ten, business_type_input)

    # Lookup the TIER value based on FINRES and RELATIONSHIP
    tier_row = tier_table[(tier_table['FINRES'] == finres_value) & (tier_table['RELATIONSHIP'] == relationship_value)]
    tier_value = tier_row['TIER'].values[0] 

    return tier_value



def show_tier_gui():
    def calculate_tier():
        try:
            # Get input values from GUI
            npc_input = npc_entry_combo.get()
            ins_score_input = ins_score_entry_combo.get()
            pc_col = pc_col_entry_combo.get()
            enterprise_col = enterprise_col_entry_combo.get()
            enterprise_ten = enterprise_ten_entry_combo.get()
            business_type_input = business_type_entry_combo.get()

            # Calculate the tier
            tier_value = tier(npc_input, ins_score_input, pc_col, enterprise_col, enterprise_ten, business_type_input)

            # Display the result
            result_label.config(text=f"TIER Value: {tier_value}")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    # Set up the window
    window = tk.Tk()
    window.title("TIER Calculation")
    window.geometry("800x400")

    # Input fields for the TIER calculation
    tk.Label(window, text="USAA Group Non-payment Cancellations (NPC)").grid(row=0, column=0, padx=10, pady=10)
    npc_entry_combo = ttk.Combobox(window, values=['0', '1', '2', '3 or more'])
    npc_entry_combo.set("3 or more") 
    npc_entry_combo.grid(row=0, column=1)

    tk.Label(window, text="Insurance Score (ChoicePoint AttractTM One)").grid(row=1, column=0, padx=10, pady=10)
    ins_score_entry_combo =  ttk.Combobox(window, values=['Range 1 (901-997)','Range 2 (861-900)','Range 3 (821-860)','Range 4 (801-820)',
 'Range 5 (781-800)', 'Range 6 (761-780)', 'Range 7 (721-760)', 'Range 8 (701-720)', 'Range 9 (681-700)',
 'Range 10 (661-680)', 'Range 10 (661-680)', 'Range 11 (641-660)', 'Range 11 (641-660)', 'Range 12 (621-640)',
 'Range 12 (621-640)', 'Range 13 (601-620)', 'Range 13 (601-620)', 'Range 14 (581-600)', 'Range 14 (581-600)',
 'Range 15 (561-580)', 'Range 15 (561-580)', 'Range 16 (541-560)', 'Range 16 (541-560)', 'Range 17 (521-540)',
 'Range 17 (521-540)', 'Range 18 (501-520)', 'Range 18 (501-520)', 'Range 19 (1-500)', 'Range 19 (1-500)',
 'Credit Score Unavailable', 'No Hit', 'No Score'], width=30)
    ins_score_entry_combo.set("Range 8 (701-720)") 
    ins_score_entry_combo.grid(row=1, column=1)

    tk.Label(window, text="P & C Collateral (Non-Auto, Non-Homeowners)").grid(row=2, column=0, padx=10, pady=10)
    pc_col_entry_combo = ttk.Combobox(window, values=[
    'Renters, Umbrella plus 2 other P&C', 'Renters, Umbrella plus 1 Other P&C',
    'Umbrella plus 2 other P&C', 'Umbrella plus 1 other P&C', 'Renters and Umbrella', 'Umbrella only',
    'Renters plus 2 other P&C', 'Renters plus 1 other P&C', '2 Other P & C lines of business',
    '1 P&C Line of Business', 'Renters only', 'No P&C Lines of Business'], width=30)
    pc_col_entry_combo.set("Umbrella plus 1 other P&C")
    pc_col_entry_combo.grid(row=2, column=1)


    tk.Label(window, text="Enterprise Collateral").grid(row=3, column=0, padx=10, pady=10)
    enterprise_col_entry_combo = ttk.Combobox(window, values=[
    'Bank, Life & IMCO', 'Bank & Life', 'Bank & IMCO', 'IMCO & Life',
    'USAA Federal Savings Bank Product (Bank)', 'USAA Life Company Product (Life)',
    'USAA Investment Management Company Product (IMCO)', 'No other Enterprise lines of business'], width=50)
    enterprise_col_entry_combo.set("Bank & IMCO")
    enterprise_col_entry_combo.grid(row=3, column=1)

    tk.Label(window, text="Enterprise Tenure").grid(row=4, column=0, padx=10, pady=10)
    enterprise_ten_entry_combo = ttk.Combobox(window, values= ['40 Years or more',
 '39 Years', '38 Years', '37 Years', '36 Years', '35 Years', '34 Years', '33 Years', '32 Years', '31 Years',
 '30 Years', '29 Years', '28 Years', '27 Years', '26 Years', '25 Years', '24 Years', '23 Years',
 '22 Years', '21 Years', '20 Years', '19 Years', '18 Years', '17 Years', '16 Years', '15 Years', '14 Years', '13 Years', '12 Years', '11 Years', '10 Years',
 '9 Years', '8 Years', '7 Years', '6 Years', '5 Years', '4 Years', '3 Years', '2 Years', '1 Year', 'Less than 1 Year', 'None'])
    enterprise_ten_entry_combo.set("35 Years")
    enterprise_ten_entry_combo.grid(row=4, column=1)

    tk.Label(window, text="Business Type").grid(row=5, column=0, padx=10, pady=10)
    business_type_entry_combo = ttk.Combobox(window, values = ['New', 'Existing', 'Renewal ', 'Returning'])
    business_type_entry_combo.set("Existing")
    business_type_entry_combo.grid(row=5, column=1)

    # Button to calculate TIER
    calculate_button = tk.Button(window, text="Calculate TIER", command=calculate_tier)
    calculate_button.grid(row=6, column=0, columnspan=2, pady=20)

    # Label to display the result
    result_label = tk.Label(window, text="TIER Value: ")
    result_label.grid(row=7, column=0, columnspan=2, pady=10)

    # Run the window
    window.mainloop()

# Show the GUI
show_tier_gui()


